## UCLA Extension Data Science Intensive 2022 Q4 Fall
Instructor: William Yu

Project: Capstone

Author: Patrick McBride

In [ ]:
#!/usr/bin/env python

# !pip3 install pandas
# !pip3 install sodapy
# !pip3 install autocensus

# Install Packages
import os
import numpy as np
import pandas as pd
from sodapy import Socrata   # https://dev.socrata.com/consumers/getting-started.html
from autocensus import Query # https://pypi.org/project/autocensus/0.2.1/

# Set working directory
os.chdir('/Users/patrick/Desktop/2022 Q4 Fall MSBA/UCLA/data/')  

# Source Data

#### Downloads can take up to 10 minutes

In [ ]:
#
# SF Data - Police Department Incident Reports: 2018 to Present
# 

# https://data.sfgov.org/Public-Safety/Police-Department-Incident-Reports-2018-to-Present/wg3w-h783

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.sfgov.org", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.sfgov.org,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
# results = client.get("wg3w-h783", limit=2000)
results = client.get_all("wg3w-h783")

# Convert to pandas DataFrame
sfcrime = pd.DataFrame.from_records(results)

# Rename Neighborhood
sfcrime = sfcrime.rename(columns = {'analysis_neighborhood' : 'nhood'})

# Save locally
sfcrime.to_csv('sfcrime.csv')

In [ ]:
#
# SF Data - San Francisco Analysis Neighborhoods
# 

# https://data.sfgov.org/dataset/San-Francisco-Analysis-Neighborhoods/xfcw-9evu

client = Socrata("data.sfgov.org", None)

# Download data
results = client.get_all("xfcw-9evu")

# Convert to pandas DataFrame
sfnhood = pd.DataFrame.from_records(results)

# Save locally
sfnhood.to_csv('sfnhood.csv')

In [ ]:
#
# SF Data - Analysis Neighborhoods - 2020 census tracts assigned to neighborhoods
# 

# Mapping of Census ACS Tract to SF Police Analysis Neighborhoods

# https://data.sfgov.org/Geographic-Locations-and-Boundaries/Analysis-Neighborhoods-2020-census-tracts-assigned/sevw-6tgi

client = Socrata("data.sfgov.org", None)

# Download data
results = client.get_all("sevw-6tgi")

# Convert to pandas DataFrame
sfnhood_tract = pd.DataFrame.from_records(results)

# Rename Neighborhood Column
sfnhood_tract = sfnhood_tract.rename(columns = {'neighborhoods_analysis_boundaries' : 'nhood'})

# Convert column data type
sfnhood_tract['tractce'] = sfnhood_tract['tractce'].astype(np.int64)

# Save locally
sfnhood_tract.to_csv('sfnhood_tract.csv')

In [ ]:
#
# US Census American Community Survey (ACS)
#

# San Francisco County demographic data by US Census Tract

#
# State (06): California
# County (075): San Francisco County
#

# Query ACS 5 Year 2020 Data
query = Query(
    estimate = 5,
    years = [2020],
    variables=[
        "DP02_0060PE", # Less than 9th grade
        "DP02_0061PE", # 9th to 12th grade, no diploma
        "DP02_0062PE", # High school graduate (includes equivalency)
        "DP02_0063PE", # Some college, no degree
        "DP02_0064PE", # Associate's degree
        "DP02_0065PE", # Bachelor's degree
        "DP02_0066PE", # Graduate or professional degree
        "DP02_0067PE", # High school graduate or higher
        "DP02_0068PE", # Bachelor's degree or higher
        "DP03_0003PE", # Percent!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force
        "DP03_0009PE", # Percent!!EMPLOYMENT STATUS!!Civilian labor force!!Unemployment Rate
        "DP03_0062E",  # Median household income (dollars) Estimate
        "DP03_0119PE", # Percentage of families and people whose income in the past 12 months is below poverty level
        "DP04_0037E",  # Estimate!!ROOMS!!Total housing units!!Median rooms
        "DP04_0047PE", # Percent!!HOUSING TENURE!!Occupied housing units!!Renter-occupied
        "DP04_0049E",  # Estimate!!HOUSING TENURE!!Occupied housing units!!Average household size of renter-occupied unit
        "DP04_0134E",  # Estimate!!GROSS RENT!!Occupied units paying rent!!Median (dollars)
        "DP04_0089E",  # Estimate!!VALUE!!Owner-occupied units!!Median (dollars)
        "DP05_0001E",  # Total population
        "DP05_0071PE", # Hispanic or Latino (of any race)
        "DP05_0077PE", # Not Hispanic or Latino; White alone
        "DP05_0078PE", # Not Hispanic or Latino; Black or African American alone
        "DP05_0079PE", # Not Hispanic or Latino; American Indian and Alaska Native alone
        "DP05_0080PE", # Not Hispanic or Latino; Asian alone
        "DP05_0081PE", # Not Hispanic or Latino; Native Hawaiian and Other Pacific Islander
        "DP05_0082PE", # Not Hispanic or Latino; Some other race alone
        "DP05_0083PE", # Not Hispanic or Latino; Two or more races
        "DP05_0005PE", # Under 5 years
        "DP05_0006PE", # 5 to 9 years
        "DP05_0007PE", # 10 to 14 years
        "DP05_0008PE", # 15 to 19 years
        "DP05_0009PE", # 20 to 24 years
        "DP05_0010PE", # 25 to 34 years
        "DP05_0011PE", # 35 to 44 years
        "DP05_0012PE", # 45 to 54 years
        "DP05_0013PE", # 55 to 59 years
        "DP05_0014PE", # 60 to 64 years
        "DP05_0015PE", # 65 to 74 years
        "DP05_0016PE", # 75 to 84 years
        "DP05_0017PE", # 85 years and over
        "DP05_0018E"], # Median age (years)
    for_geo = ['tract:*'],
    in_geo = ['state:06', 'county:075'],
    # Fill in the following with your own Census API key; https://api.census.gov/data/key_signup.html
    census_api_key = 'a792cb3572562c0f7d01d3eb0ecea17bf00f977c'
)

# Run query and collect output in dataframe
sfacs5y2020 = query.run()

# Parse State, County, and Tract Geography ID from geo_id for mapping with SF Neighborhoods
sfacs5y2020['state_fp'] = sfacs5y2020['geo_id'].str[9:11]
sfacs5y2020['county_fp'] = sfacs5y2020['geo_id'].str[11:14]
sfacs5y2020['tractce'] = sfacs5y2020['geo_id'].str[14:len(sfacs5y2020['geo_id'])].astype(np.int64)

# Save locally
sfacs5y2020.to_csv('sfacs5y2020.csv')

In [ ]:
#
# SF Data - Reference: Police Department Incident Code Crosswalk
# 

#################################################################################
# Not used!!!!                                                                  #
#################################################################################

# https://data.sfgov.org/Public-Safety/Reference-Police-Department-Incident-Code-Crosswal/ci9u-8awy

# client = Socrata("data.sfgov.org", None)

# # Download data
# results = client.get_all("ci9u-8awy")
# # Convert to pandas DataFrame
# sfcrime_code = pd.DataFrame.from_records(results)

# # Save locally
# sfcrime_code.to_csv('sfcrime_code.csv')

In [ ]:
#
# SF Data - Police Department Incident Reports: Historical 2003 to May 2018
# 

#################################################################################
# Not used!!!!                                                                  #
#################################################################################

# https://data.sfgov.org/Public-Safety/Police-Department-Incident-Reports-Historical-2003/tmnf-yvry

# client = Socrata("data.sfgov.org", None)

# # Download data
# results = client.get_all("tmnf-yvry")

# # Convert to pandas DataFrame
# sfcrime2003 = pd.DataFrame.from_records(results)

# # Save locally
# sfcrime2003.to_csv('sfcrime2003.csv')

# US Census Data Profile Codes

In [ ]:
# 
# US Census America Community Survey (ACS) Overview
#

# https://data.census.gov/table?tid=ACSDP5Y2020.DP05&g=0400000US06_0500000US06075

# Topics currently included with autocensus are population, race, education, income, and housing.

# query = Query(
#     estimate=5,
#     years=[2013, 2014, 2015, 2016, 2017],
#     # Housing variables: B25035_001E, B25064_001E, B25077_001E
#     variables=autocensus.topics.housing,
#     for_geo='tract:*',
#     in_geo=['state:06', 'county:075']
# )


#
# DP02 - Social
#
# Education Attainment; Percentage Population 25 years and over
"DP02_0060PE" : "ed_below9" # Less than 9th grade
"DP02_0061PE" : "ed_g912" # 9th to 12th grade, no diploma
"DP02_0062PE" : "ed_hs" # High school graduate (includes equivalency)
"DP02_0063PE" : "ed_scollege" # Some college, no degree
"DP02_0064PE" : "ed_associate" # Associate's degree
"DP02_0065PE" : "ed_bachelor" # Bachelor's degree
"DP02_0066PE" : "ed_higher" # Graduate or professional degree
"DP02_0067PE" : "ed_hs_higher" # High school graduate or higher
"DP02_0068PE" : "ed_college_higher" # Bachelor's degree or higher


#
# DP03 - Economic
#
"DP03_0003PE" : "eco_clf" # Percent!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force
"DP03_0009PE" : "eco_unemp" # Percent!!EMPLOYMENT STATUS!!Civilian labor force!!Unemployment Rate
"DP03_0062E"  : "eco_med_hincome" # Median household income (dollars) Estimate
"DP03_0119PE" : "eco_poverty"     # Percentage of families and people whose income in the past 12 months is below poverty level
"DP03_0032PE" : "eco_emp" # Civilian employed population 16 years and over


#
# DP04 - Housing
#
"DP04_0037E" : "h_med_rooms"     # Estimate!!ROOMS!!Total housing units!!Median rooms
"DP04_0047PE" : "h_rental_units" # Percent!!HOUSING TENURE!!Occupied housing units!!Renter-occupied
"DP04_0049E" : "h_rentel_size"   # Estimate!!HOUSING TENURE!!Occupied housing units!!Average household size of renter-occupied unit
"DP04_0134E" : "h_med_rent"      # Estimate!!GROSS RENT!!Occupied units paying rent!!Median (dollars)
"DP04_0089E" : "h_med_homeprice" # Estimate!!VALUE!!Owner-occupied units!!Median (dollars)


#
# DP05 - Demographic
#
# https://api.census.gov/data/2020/acs/acs5/profile/groups/DP05.html
"DP05_0001E"  : "pop"         # Total Population
"DP05_0018E"  : "mage" # Median age (years)

# Total Population; Race Percentage (Tally 100%)
"DP05_0037PE" : "pop_white"   # White
"DP05_0038PE" : "pop_black"   # Black or African American
"DP05_0039PE" : "pop_aindina" # American Indian and Alaska Native
"DP05_0044PE" : "pop_asian"   # Asian
"DP05_0052PE" : "pop_pacific" # Native Hawaiian and Other Pacific Islander
"DP05_0057PE" : "pop_other"   # Some other race
"DP05_0058PE" : "pop_twoplus" # Two or more races

# Total Population; Race Hispanic or Latino Percentage (Tally 100%)
"DP05_0071PE" : "poph_latino"  # Hispanic or Latino (of any race)
"DP05_0077PE" : "poph_white"   # Not Hispanic or Latino; White alone
"DP05_0078PE" : "poph_black"   # Not Hispanic or Latino; Black or African American alone
"DP05_0079PE" : "poph_aindina" # Not Hispanic or Latino; American Indian and Alaska Native alone
"DP05_0080PE" : "poph_asian"   # Not Hispanic or Latino; Asian alone
"DP05_0081PE" : "poph_pacific" # Not Hispanic or Latino; Native Hawaiian and Other Pacific Islander
"DP05_0082PE" : "poph_other"   # Not Hispanic or Latino; Some other race alone
"DP05_0083PE" : "poph_twoplus" # Not Hispanic or Latino; Two or more races

# Total Population; Age Percentage (Tally 100%)
"DP05_0005PE" : "ab5"   # Under 5 years
"DP05_0006PE" : "a59"   # 5 to 9 years
"DP05_0007PE" : "a1014" # 10 to 14 years
"DP05_0008PE" : "a1519" # 15 to 19 years
"DP05_0009PE" : "a2024" # 20 to 24 years
"DP05_0010PE" : "a2534" # 25 to 34 years
"DP05_0011PE" : "a3544" # 35 to 44 years
"DP05_0012PE" : "a4554" # 45 to 54 years
"DP05_0013PE" : "a5559" # 55 to 59 years
"DP05_0014PE" : "a6064" # 60 to 64 years
"DP05_0015PE" : "a6574" # 65 to 74 years
"DP05_0016PE" : "a7584" # 75 to 84 years
"DP05_0017PE" : "a85a" # 85 years and over

# Load Data (no need to download each time)

In [ ]:
#
# Quickly start by loading instead of downloading from source
# 
sfcrime = pd.read_csv('sfcrime.csv')
sfnhood = pd.read_csv('sfnhood.csv')
sfnhood_tract = pd.read_csv('sfnhood_tract.csv')
sfacs5y2020 = pd.read_csv('sfacs5y2020.csv')

# Exploratory Data Analysis (EDA)

## Data Cleanup- SF Crime

In [ ]:
sfcrime.info()

### Missing Data - SF Crime

In [ ]:
# !pip3 install missingno
import missingno as msno

msno.matrix(sfcrime.sample(1000))
# msno.heatmap(sfcrime)
# msno.dendrogram(sfcrime)

### SF Map by Neighborhood, Police District, Supervisor District, Tract

### Key Variables - SF Crime

In [ ]:
sfcrime = sfcrime[['incident_datetime',
    'incident_id',
    'incident_number',
    'incident_code',
    'incident_category',
    'incident_subcategory',
    'incident_description',
    'nhood',
    'police_district',
    'supervisor_district',
    'intersection',
    'longitude',
    'latitude']]

### Remove Duplicates - SF Crime

In [ ]:
sfcrime.duplicated().sum()

In [ ]:
sfcrime = sfcrime.drop_duplicates()

### Remove NaN - SF Crime

#### Incident (Sub)Category (~500) or Location (Out of SF) (~35k) with NaN

In [ ]:
sfcrime.isna().sum()

In [ ]:
sfcrime = sfcrime.dropna()

In [ ]:
msno.matrix(sfcrime.sample(1000))

### Clean up Misspellings

In [ ]:
# sfcrime.loc[(sfcrime['incident_category'] == 'Weapons Offence') ,'incident_category'] = 'Weapons Offense'

### Remove Case Closures

In [ ]:
len(sfcrime[sfcrime['incident_category'] == 'Case Closure'])

In [ ]:
sfcrime = sfcrime[sfcrime['incident_category'] != 'Case Closure']

In [ ]:
#
# Multiple Incident Codes
# 
# https://datasf.gitbook.io/datasf-dataset-explainers/sfpd-incident-report-2018-to-present

# Incident reports can have one or more associated Incident Codes
# For example, an officer may have a warrant for an arrest and while making the arrest, 
# discovers narcotics in the individual’s possession. 
# The officer would record two Incident Codes: (1) for the warrant and (2) for the discovery of narcotics.
sfcrime.groupby(by = ['incident_number']).filter(lambda x: len(x) > 1)

In [ ]:
sfcrime.info()

In [ ]:
sfcrime.nunique()

### Introduce Time Series - SF Crime

In [ ]:
import datetime

sfcrime['incident_datetime'] = pd.to_datetime(sfcrime['incident_datetime'])
sfcrime['incident_date'] = pd.to_datetime(sfcrime['incident_datetime'].dt.date)
sfcrime['incident_year'] = pd.to_datetime(sfcrime['incident_datetime'].dt.strftime('%Y-01-01'))
sfcrime['incident_yearmonth'] = pd.to_datetime(sfcrime['incident_datetime'].dt.strftime('%Y-%m-01'))
sfcrime['incident_monthofyear'] = sfcrime['incident_datetime'].dt.month
sfcrime['incident_dayofmonth'] = sfcrime['incident_datetime'].dt.day
sfcrime['incident_dayofweek'] = sfcrime['incident_datetime'].dt.dayofweek
sfcrime['incident_hour'] = sfcrime['incident_datetime'].dt.hour
sfcrime['incident_minute'] = sfcrime['incident_datetime'].dt.minute


## Data Cleanup - SF ACS

In [ ]:
sfacs5y2020.info()

### Missing Data - SF ACS

In [ ]:
msno.matrix(sfacs5y2020.sample(1000))

### Remove Duplicates - SF ACS

In [ ]:
#
# Remove annotation column
# 
sfacs5y2020 = sfacs5y2020.drop(columns = 'annotation')

In [ ]:
sfacs5y2020.duplicated().sum()

In [ ]:
sfacs5y2020 = sfacs5y2020.drop_duplicates()

### Remove NaN - SF ACS

In [ ]:
sfacs5y2020.isna().sum()

In [ ]:
sfacs5y2020 = sfacs5y2020.dropna()

In [ ]:
msno.matrix(sfacs5y2020)

In [ ]:
sfacs5y2020.info()

In [ ]:
sfacs5y2020.nunique()

In [ ]:
sfnhood_tract.info()

## Map US Census Tract Demographic to SF Crime Analysis Neighborhood

In [ ]:
#
# Merge SF Crime Analysis Neighborhoods with Census Tract demograpics
#
sfnhood_acs5y2020 = pd.merge(
    sfnhood_tract[['nhood', 'tractce']], 
    sfacs5y2020[['variable_code', 'value', 'tractce']], 
    on = ["tractce"]
    )

In [ ]:
# Tally Neighborhood Incidents by Demographic Census Code
sfnhood_acs5y2020 = sfnhood_acs5y2020.groupby(
    by = ['nhood', 'variable_code']
    ).aggregate({'value' : 'mean'}
    ).reset_index()

In [ ]:
# Pivot from long (demographic code) to wide (neighborhood)
sfnhood_acs5y2020 = sfnhood_acs5y2020.pivot(
    index = 'nhood', 
    columns = 'variable_code', 
    values = 'value'
    ).reset_index()

In [ ]:
#
# Calculate Crime Incident Categories Numbers by Neighborhood
#
sfcrime_nhood = sfcrime.groupby(
    by = ['nhood', 'incident_category']
    ).aggregate(incidents = ('incident_category', 'count')
    ).reset_index()

In [ ]:
# Pivot long (incident category) to wide (neighborhood)
sfcrime_nhood = sfcrime_nhood.pivot(
    index = 'nhood', 
    columns = 'incident_category', 
    values = 'incidents'
    ).reset_index()

In [ ]:
# Replace NaN with zero (0)
sfcrime_nhood = sfcrime_nhood.fillna(0)

In [ ]:
#
# Merge Neighborhood Crime
# 
sfnhood_acs5y2020 = pd.merge(
    sfnhood_acs5y2020, 
    sfcrime_nhood, 
    on = ['nhood'])

# Introduce Neighborhood Identifier
sfnhood_acs5y2020["nhood_id"] = sfnhood_acs5y2020.index + 1

In [ ]:
#
# Calculate Crime Incidents by Neighborhood
# 
sfnhood_acs5y2020 = pd.merge(
    sfnhood_acs5y2020, 
    sfcrime.groupby(by = ['nhood']).aggregate(incidents = ('incident_code', 'count')),
    on = ['nhood'])  

In [ ]:
#
# Merge SF Analysis Neighborhood with Geographic Data
#
sfnhood_acs5y2020 = pd.merge(
    sfnhood_acs5y2020, 
    sfnhood[['the_geom', 'nhood']], 
    on = ['nhood'])

In [ ]:
sfnhood_acs5y2020 = sfnhood_acs5y2020.fillna(0)

### Rename US Census ACS Columns

In [ ]:
sfnhood_acs5y2020 = sfnhood_acs5y2020.rename(columns = {
    "DP02_0060PE" : "ed_below9", # Less than 9th grade
    "DP02_0061PE" : "ed_g912", # 9th to 12th grade, no diploma
    "DP02_0062PE" : "ed_hs", # High school graduate (includes equivalency)
    "DP02_0063PE" : "ed_scollege", # Some college, no degree
    "DP02_0064PE" : "ed_associate", # Associate's degree
    "DP02_0065PE" : "ed_bachelor", # Bachelor's degree
    "DP02_0066PE" : "ed_higher", # Graduate or professional degree
    "DP02_0067PE" : "ed_hs_higher", # High school graduate or higher
    "DP02_0068PE" : "ed_college_higher", # Bachelor's degree or higher
    "DP03_0003PE" : "eco_clf", # Percent!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force
    "DP03_0009PE" : "eco_unemp", # Percent!!EMPLOYMENT STATUS!!Civilian labor force!!Unemployment Rate
    "DP03_0062E"  : "eco_med_hincome", # Median household income (dollars) Estimate
    "DP03_0119PE" : "eco_poverty",     # Percentage of families and people whose income in the past 12 months is below poverty level
    "DP04_0037E" : "h_med_rooms", # Estimate!!ROOMS!!Total housing units!!Median rooms
    "DP04_0047PE" : "h_rental_units", # Percent!!HOUSING TENURE!!Occupied housing units!!Renter-occupied
    "DP04_0049E" : "h_rentel_size", # Estimate!!HOUSING TENURE!!Occupied housing units!!Average household size of renter-occupied unit
    "DP04_0134E" : "h_med_rent", # Estimate!!GROSS RENT!!Occupied units paying rent!!Median (dollars)
    "DP04_0089E" : "h_med_homeprice", # Estimate!!VALUE!!Owner-occupied units!!Median (dollars)
    "DP05_0001E"  : "pop",              # Total Population
    "DP05_0071PE" : "poph_latino",  # Hispanic or Latino (of any race)
    "DP05_0077PE" : "poph_white",   # Not Hispanic or Latino; White alone
    "DP05_0078PE" : "poph_black",   # Not Hispanic or Latino; Black or African American alone
    "DP05_0079PE" : "poph_aindian", # Not Hispanic or Latino; American Indian and Alaska Native alone
    "DP05_0080PE" : "poph_asian",   # Not Hispanic or Latino; Asian alone
    "DP05_0081PE" : "poph_pacific", # Not Hispanic or Latino; Native Hawaiian and Other Pacific Islander
    "DP05_0082PE" : "poph_other",   # Not Hispanic or Latino; Some other race alone
    "DP05_0083PE" : "poph_twoplus", # Not Hispanic or Latino; Two or more races
    "DP05_0005PE" : "ab5",   # Under 5 years
    "DP05_0006PE" : "a59",   # 5 to 9 years
    "DP05_0007PE" : "a1014", # 10 to 14 years
    "DP05_0008PE" : "a1519", # 15 to 19 years
    "DP05_0009PE" : "a2024", # 20 to 24 years
    "DP05_0010PE" : "a2534", # 25 to 34 years
    "DP05_0011PE" : "a3544", # 35 to 44 years
    "DP05_0012PE" : "a4554", # 45 to 54 years
    "DP05_0013PE" : "a5559", # 55 to 59 years
    "DP05_0014PE" : "a6064", # 60 to 64 years
    "DP05_0015PE" : "a6574", # 65 to 74 years
    "DP05_0016PE" : "a7584", # 75 to 84 years
    "DP05_0017PE" : "a85a", # 85 years and over
    "DP05_0018E"  : "mage" # Median age (years)
})


### Calculate City Human Capital Index (CHCI)

In [ ]:
#
# Calculate CHCI
# 
# https://www.anderson.ucla.edu/about/centers/ucla-anderson-forecast/projects-and-partnerships/city-human-capital-index

sfnhood_acs5y2020['chci'] = ((1 / 100) * 
    (50 * sfnhood_acs5y2020['ed_below9'] + 
    100 * sfnhood_acs5y2020['ed_g912'] +
    120 * sfnhood_acs5y2020['ed_hs'] +
    130 * sfnhood_acs5y2020['ed_scollege'] +
    140 * sfnhood_acs5y2020['ed_associate'] +
    190 * sfnhood_acs5y2020['ed_bachelor'] +
    230 * sfnhood_acs5y2020['ed_higher']))

#
# Calculate Education (Tally 100%)
#
# sfnhood_acs5y2020['ed'] = (sfnhood_acs5y2020['ed_below9'] +
#     sfnhood_acs5y2020['ed_g912'] +
#     sfnhood_acs5y2020['ed_hs'] +
#     sfnhood_acs5y2020['ed_scollege'] +
#     sfnhood_acs5y2020['ed_associate'] +
#     sfnhood_acs5y2020['ed_bachelor'] +
#     sfnhood_acs5y2020['ed_higher'])

#
# Calculate Population (Tally 100%)
#
# sfnhood_acs5y2020['pop%'] = (sfnhood_acs5y2020['poph_latino'] +
#     sfnhood_acs5y2020['poph_white'] +
#     sfnhood_acs5y2020['poph_black'] +
#     sfnhood_acs5y2020['poph_aindina'] +
#     sfnhood_acs5y2020['poph_asian'] +
#     sfnhood_acs5y2020['poph_pacific'] +
#     sfnhood_acs5y2020['poph_other'] +
#     sfnhood_acs5y2020['poph_twoplus'])

## Analyze Crime Data by Category

In [ ]:

import seaborn as sns # Import seaborn
import matplotlib.pyplot as plt

# Set seaboard theme
ax = sns.set(style='darkgrid')

In [ ]:
#
# SF Crime Category Hierarchy:
#
#  Category 
#      - Subcategory 
#            - Description

sfcrime.groupby(by = ['incident_category','incident_subcategory','incident_description']).size()

In [ ]:
#
# SF Crime Category
# 
sfcrime_cat_freq = sfcrime.groupby(by = ['incident_category']
    ).aggregate(incidents = ('incident_code', 'count')
    ).sort_values(by = 'incidents', ascending = False
    ).reset_index()
sfcrime_cat_freq['frequency'] = sfcrime_cat_freq['incidents']/sfcrime_cat_freq['incidents'].sum()
sfcrime_cat_freq['cum_frequency'] = sfcrime_cat_freq['frequency'].cumsum()

#
# Plot - All Categories
#
fig, ax = plt.subplots(figsize=(16, 9))

ax = sns.barplot(
    data = sfcrime_cat_freq,
    y = 'incident_category',
    x = 'incidents',
    palette="Reds_r")

ax.set_xlabel('Number of Incidents',fontsize=20)
ax.set_ylabel('Incident Category',fontsize=20)
ax.set_title('2018 to Present: Incidents by Category', fontsize=20)

In [ ]:
#
# Plot - Top X Categories makes up X% of incidents
#
topn = 10

#
# Plot - All Categories
#
fig, ax = plt.subplots(figsize=(16, 9))

ax = sns.barplot(
    data = sfcrime_cat_freq.iloc[:topn],
    y = 'incident_category',
    x = 'incidents',
    palette="Reds_r")

ax.set_xlabel('Number of Incidents',fontsize=20)
ax.set_ylabel('Incident Category',fontsize=20)
ax.set_title('2018 to Present: {:0.0f}% Incidents by Top {} Categories'.format(
    sfcrime_cat_freq['cum_frequency'].iloc[topn] * 100,
    topn), 
    fontsize=20)

In [ ]:
#
# SF Crime Subategory
# 
sfcrime_subcat_freq = sfcrime.groupby(by = ['incident_subcategory']
    ).aggregate(incidents = ('incident_code', 'count')
    ).sort_values(by = 'incidents', ascending = False
    ).reset_index()
sfcrime_subcat_freq['frequency'] = sfcrime_subcat_freq['incidents']/sfcrime_subcat_freq['incidents'].sum()
sfcrime_subcat_freq['cum_frequency'] = sfcrime_subcat_freq['frequency'].cumsum()

#
# Plot
#
fig, ax = plt.subplots(figsize=(16, 9))

ax = sns.barplot(
    data = sfcrime_subcat_freq,
    y = 'incident_subcategory',
    x = 'incidents',
    palette="Reds_r")

ax.set_xlabel('Number of Incidents',fontsize=20)
ax.set_ylabel('Incident Subcategory',fontsize=20)
ax.set_title('2018 to Present: Incidents by Subcategory', fontsize=20)

In [ ]:
#
# SF Crime Subategory - Top 17 / 71 (24%) makes up 82% of incidents
# 
topn = 17

#
# Plot
#
fig, ax = plt.subplots(figsize=(16, 9))

ax = sns.barplot(
    data = sfcrime_subcat_freq.iloc[:topn],
    y = 'incident_subcategory',
    x = 'incidents',
    palette="Reds_r")

ax.set_xlabel('Number of Incidents',fontsize=20)
ax.set_ylabel('Incident Subcategory',fontsize=20)
ax.set_title('2018 to Present: {:0.0f}% Incidents by Top {} Subcategories'.format(
    sfcrime_subcat_freq['cum_frequency'].iloc[topn] * 100,
    topn), 
    fontsize=20)

In [ ]:
#
# SF Crime Description
# 
sfcrime_desc_freq = sfcrime.groupby(by = ['incident_description']
    ).aggregate(incidents = ('incident_code', 'count')
    ).sort_values(by = 'incidents', ascending = False
    ).reset_index()
sfcrime_desc_freq['frequency'] = sfcrime_desc_freq['incidents']/sfcrime_desc_freq['incidents'].sum()
sfcrime_desc_freq['cum_frequency'] = sfcrime_desc_freq['frequency'].cumsum()

#
# Plot
#
fig, ax = plt.subplots(figsize=(16, 9))

ax = sns.barplot(
    data = sfcrime_desc_freq,
    y = 'incident_description',
    x = 'incidents',
    palette="Reds_r")

ax.set_xlabel('Number of Incidents',fontsize=20)
ax.set_ylabel('Incident Description',fontsize=20)
ax.set_title('2018 to Present: Incidents by Description', fontsize=20)

In [ ]:
#
# SF Crime Description - Top 20 Descriptions
# 
topn = 20

#
# Plot
#
fig, ax = plt.subplots(figsize=(16, 9))

ax = sns.barplot(
    data = sfcrime_desc_freq.iloc[:topn],
    y = 'incident_description',
    x = 'incidents',
    palette="Reds_r")

ax.set_xlabel('Number of Incidents',fontsize=20)
ax.set_ylabel('Incident Description (Code)',fontsize=20)
ax.set_title('2018 to Present: {:0.0f}% Incidents by Top {} Descriptions'.format(
    sfcrime_desc_freq['cum_frequency'].iloc[topn] * 100,
    topn), 
    fontsize=20)

## Analyze Crime Data by Location

In [ ]:
#
# SF Crime Intersection - Top 31 / 6371 (3%) makes up 10% of incidents
# 
sfcrime_intersecation_freq = sfcrime.groupby(
    by = ['intersection']
    ).aggregate(incidents = ('incident_code', 'count')
    ).sort_values(by = 'incidents', ascending = False
    ).reset_index()
sfcrime_intersecation_freq['frequency'] = sfcrime_intersecation_freq['incidents']/sfcrime_intersecation_freq['incidents'].sum()
sfcrime_intersecation_freq['cum_frequency'] = sfcrime_intersecation_freq['frequency'].cumsum()

topn = 30

#
# Plot
#
fig, ax = plt.subplots(figsize=(16, 9))

ax = sns.barplot(
    data = sfcrime_intersecation_freq.iloc[:topn],
    y = 'intersection',
    x = 'incidents',
    palette="Reds_r")

ax.set_xlabel('Number of Incidents',fontsize=20)
ax.set_ylabel('Incident Intersections',fontsize=20)
ax.set_title('2018 to Present: {:0.0f}% Incidents by Top {} Intersections'.format(
    sfcrime_intersecation_freq['cum_frequency'].iloc[topn] * 100,
    topn), 
    fontsize=20)

In [ ]:
#
# SF Crime Analysis Neighborhood
# 
sfcrime_nhood_freq = sfcrime.groupby(by = ['nhood']
    ).aggregate(incidents = ('incident_code', 'count')
    ).sort_values(by = 'incidents', ascending = False
    ).reset_index()
sfcrime_nhood_freq['frequency'] = sfcrime_nhood_freq['incidents']/sfcrime_nhood_freq['incidents'].sum()
sfcrime_nhood_freq['cum_frequency'] = sfcrime_nhood_freq['frequency'].cumsum()

# Plot
fig, ax = plt.subplots(figsize=(16, 9))

ax = sns.barplot(
    data = sfcrime_nhood_freq,
    y = 'nhood',
    x = 'incidents',
    palette="Reds_r")

ax.set_xlabel('Number of Incidents',fontsize=20)
ax.set_ylabel('Incident Neighborhood',fontsize=20)
ax.set_title('2018 to Present: Incidents by Neighborhood', fontsize=20)

In [ ]:
#
# SF Crime Analysis Neighborhood - Top 6 / 41 (15%) makes up 50% of incidents
# 
topn = 6

#
# Plot
#
fig, ax = plt.subplots(figsize=(16, 9))

ax = sns.barplot(
    data = sfcrime_nhood_freq.iloc[:topn],
    y = 'nhood',
    x = 'incidents',
    palette="Reds_r")

ax.set_xlabel('Number of Incidents',fontsize=20)
ax.set_ylabel('Incident Neighborhood',fontsize=20)
ax.set_title('2018 to Present: {:0.0f}% Incidents by Top {} Neighborhood'.format(
    sfcrime_nhood_freq['cum_frequency'].iloc[topn] * 100,
    topn), 
    fontsize=20)

## Analyze Crime Data by Time

In [ ]:
import matplotlib.dates as mdates

In [ ]:
#
# Visualize Crime Incidents by Time 
#

#######################################################################
# Skewed due to incident count not mean; Need to standardized over time
#######################################################################

fig, axes = plt.subplots(2, 3, figsize=(16, 9))
 
fig.suptitle('SF Crime Incident by Time')
  
sns.barplot(ax=axes[0, 0], data = sfcrime.groupby(by = ['incident_year'])['incident_code'].count().reset_index(), x='incident_year', y='incident_code', palette="dark:blue")
axes[0, 0].set(ylabel='Number of Incidents', xlabel='Year')
axes[0, 0].set_xticklabels(axes[0, 0].get_xticklabels(), rotation=45, horizontalalignment='right')
sns.barplot(ax=axes[0, 1], data = sfcrime.groupby(by = ['incident_monthofyear'])['incident_code'].count().reset_index(), x='incident_monthofyear', y='incident_code', palette="dark:blue")
axes[0, 1].set(ylabel='', xlabel='Month of Year')
sns.barplot(ax=axes[0, 2], data = sfcrime.groupby(by = ['incident_dayofmonth'])['incident_code'].count().reset_index(), x='incident_dayofmonth', y='incident_code', palette="dark:blue")
axes[0, 2].set(ylabel='', xlabel='Day of Month')
sns.barplot(ax=axes[1, 0], data = sfcrime.groupby(by = ['incident_dayofweek'])['incident_code'].count().reset_index(), x='incident_dayofweek', y='incident_code', palette="dark:blue")
axes[1, 0].set(ylabel='', xlabel='Day of Week')
sns.barplot(ax=axes[1, 1], data = sfcrime.groupby(by = ['incident_hour'])['incident_code'].count().reset_index(), x='incident_hour', y='incident_code', palette="dark:blue")
axes[1, 1].set(ylabel='', xlabel='Hour')
sns.barplot(ax=axes[1, 2], data = sfcrime.groupby(by = ['incident_minute'])['incident_code'].count().reset_index(), x='incident_minute', y='incident_code', palette="dark:blue")
axes[1, 2].set(ylabel='', xlabel='Minute')

In [ ]:
#
# Visualize Crime Incidents by Minute of Day
#
df = sfcrime.groupby(by = ['incident_datetime']).aggregate(incidents = ('incident_datetime', 'count')).rename_axis('date')

df = df.resample("Min").mean()

#
# Crime Incidents by Minute of Day
# 
fig, ax = plt.subplots(figsize=(16, 9))

ax = sns.barplot(
    data = df,
    y = 'incidents',
    x = (df.index).minute,
    errorbar = 'sd',
    capsize = 0.2,
    palette="ch:start = .2, rot = -.2, dark = 0.4, light = 0.8"
    )
              
ax.set_xlabel('Time (Minute of Hour)',fontsize=20)
ax.set_ylabel('Average Incidents by Minute',fontsize=20)
ax.set_title('2018 to Present: Average Incidents by Minute', fontsize=20)

In [ ]:
#
# Visualize Crime Incidents by Hour of Day
#
df = sfcrime.groupby(by = ['incident_datetime']).aggregate(incidents = ('incident_datetime', 'count')).rename_axis('date')

df = df.resample("H").mean()

#
# Crime Incidents by Hour of Day
# 
fig, ax = plt.subplots(figsize=(16, 9))

ax = sns.barplot(
    data = df,
    y = 'incidents',
    x = (df.index).hour,
    errorbar = 'sd',
    capsize = 0.2,
    palette="ch:start = .2, rot = -.2, dark = 0.4, light = 0.8"
    )
              
ax.set_xlabel('Time (Hour of the day)',fontsize=20)
ax.set_ylabel('Average Hourly Incidents',fontsize=20)
ax.set_title('2018 to Present: Average Hourly Incidents by Hour of Day', fontsize=20)

In [ ]:
#
# Visualize Crime Incidents by Day of Week
#
df = sfcrime.groupby(by = ['incident_date']).aggregate(incidents = ('incident_datetime', 'count')).rename_axis('date')

df = df.resample("D").mean()

#
# Crime Incidents by Day of Week
# 
fig, ax = plt.subplots(figsize=(16, 9))

ax = sns.barplot(
   data = df,
   y = 'incidents',
   x = (df.index).strftime('%A'),
   errorbar = 'sd',
   capsize = 0.2,
   palette="ch:start = .2, rot = -.2, dark = 0.4, light = 0.8"
   )

ax.set_xlabel('Time (Day of Week)',fontsize=20)
ax.set_ylabel('Average Daily Incidents',fontsize=20)
ax.set_title('2018 to Present: Average Daily Incidents by Day of Week', fontsize=20)

In [ ]:
#
# Visualize Crime Incidents by Day of Month
#
df = sfcrime.groupby(by = ['incident_date']).aggregate(incidents = ('incident_datetime', 'count')).rename_axis('date')

df = df.resample("D").mean()

#
# Crime Incidents by Day of Month
# 
fig, ax = plt.subplots(figsize=(16, 9))

ax = sns.barplot(
   data = df,
   y = 'incidents',
   x = (df.index).day,
   errorbar = 'sd',
   capsize = 0.2,
   palette="ch:start = .2, rot = -.2, dark = 0.4, light = 0.8"
   )
              
ax.set_xlabel('Time (Day of Month)',fontsize=20)
ax.set_ylabel('Average Daily Incidents',fontsize=20)
ax.set_title('2018 to Present: Average Daily Incidents by Day of Month', fontsize=20)

In [ ]:
#
# Visualize Crime Incidents by Day
#
df = sfcrime.groupby(by = ['incident_date']).aggregate(incidents = ('incident_datetime', 'count')).rename_axis('date')

df = df.resample("M").mean()

#
# Crime Incidents by Month of Year
# 
fig, ax = plt.subplots(figsize=(16, 9))

ax = sns.barplot(
   data = df,
   y = 'incidents',
   x = (df.index).strftime('%B'),
   errorbar = 'sd',
   capsize = 0.2,
   palette="ch:start = .2, rot = -.2, dark = 0.4, light = 0.8"
   )
              
ax.set_xlabel('Time (Month of Year)',fontsize=20)
ax.set_ylabel('Average Daily Incidents',fontsize=20)
ax.set_title('2018 to Present: Average Daily Incidents by Month of Year', fontsize=20)

In [ ]:
#
# Visualize Crime Incidents by Year
#
df = sfcrime.groupby(by = ['incident_date']).aggregate(incidents = ('incident_datetime', 'count')).rename_axis('date')

df = df.resample("D").mean()

#
# Average Daily Crime Incidents by Year
# 
fig, ax = plt.subplots(figsize=(16, 9))

ax = sns.barplot(
   data = df,
   y = 'incidents',
   x = (df.index).year,
   errorbar = 'sd',
   capsize = 0.2,
   palette="ch:start = .2, rot = -.2, dark = 0.4, light = 0.8"
   )
              
ax.set_xlabel('Time (Year)',fontsize=20)
ax.set_ylabel('Average Daily Incidents',fontsize=20)
ax.set_title('2018 to Present: Average Daily Incidents by Year', fontsize=20)

In [ ]:
#
# SF Crime Time Series by Day
# 

#  Plot
fig, ax = plt.subplots(figsize=(30, 10))

ax = sns.lineplot(
  data = sfcrime.groupby(by = ['incident_date'])['incident_code'].count().reset_index(),
  x = 'incident_date', 
  y = 'incident_code'
  )

ax.set(title = 'SF Crime Incidents - 2018 to Present', ylabel='Number of Incidents', xlabel='Time')

In [ ]:
#
# Zoom into specific dates
#
fig, ax = plt.subplots(figsize=(16, 9))

ax = sns.lineplot(
  data = sfcrime[
    (sfcrime['incident_date'] > '2021-02') & 
    (sfcrime['incident_date'] < '2021-06')
    ].groupby(by = ['incident_date']
    )['incident_code'].count().reset_index(),
  x = 'incident_date',
  y = 'incident_code'
  )

ax.set(title = 'SF Crime Incidents - 2018 to Present', ylabel='Number of Incidents', xlabel='Time')

## Seasonality

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
#
# Time Series Decomposition with Daily Incidents - Additive v Multiplicative (same?)
# 

df = sfcrime.groupby(by = ['incident_date']).aggregate(value = ('incident_datetime', 'count'))

decomposition_mul = seasonal_decompose(df, model='multiplicative')
decomposition_add = seasonal_decompose(df, model='additive')

def plotseasonal(res, axes, title):
    axes[0].title.set_text(title)

    res.observed.plot(ax=axes[0], legend=False)
    axes[0].set_ylabel('Observed')
    
    res.trend.plot(ax=axes[1], legend=False)
    axes[1].set_ylabel('Trend')
    
    res.seasonal.plot(ax=axes[2], legend=False)
    axes[2].set_ylabel('Seasonal')
    
    res.resid.plot(ax=axes[3], legend=False)
    axes[3].set_ylabel('Residual')

fig, axes = plt.subplots(ncols=2, nrows=4, sharex=False, figsize=(30,15))
plotseasonal(decomposition_mul, axes[:,0], title="Multiplicative")
plotseasonal(decomposition_add, axes[:,1], title="Additive")

In [ ]:
#
# Additive Time Series Decomposition with Daily Incidents
# 
df = sfcrime.groupby(by = ['incident_date']).aggregate(value = ('incident_datetime', 'count'))

plt.rc("figure",figsize=(16, 9))
analysis = df[['value']].copy()

decompose_result = seasonal_decompose(analysis, model="additive")

observed = decompose_result.observed
trend = decompose_result.trend
seasonal = decompose_result.seasonal
residual = decompose_result.resid

decompose_result.plot();

In [ ]:
#
# Additive Time Series Decomposition with Monthly Incidents
# 
df = sfcrime.groupby(by = ['incident_yearmonth']).aggregate(value = ('incident_datetime', 'count'))

plt.rc("figure",figsize=(16,9))
analysis = df[['value']].copy()

decompose_result_mult = seasonal_decompose(analysis, model="additive")

trend = decompose_result_mult.trend
seasonal = decompose_result_mult.seasonal
residual = decompose_result_mult.resid

decompose_result_mult.plot();

In [ ]:
#
# Multiplicative Time Series Decomposition with Monthly Incidents
# 
df = sfcrime.groupby(by = ['incident_yearmonth']).aggregate(value = ('incident_datetime', 'count'))

plt.rc("figure",figsize=(16,9))
analysis = df[['value']].copy()

decompose_result = seasonal_decompose(analysis, model="multiplicative")

trend = decompose_result.trend
seasonal = decompose_result.seasonal
residual = decompose_result.resid

decompose_result.plot();

## Crime Incident Distribution

In [ ]:
ax = sns.displot(sfcrime[['incident_date']].groupby(by = ['incident_date']).value_counts(), kde = True)
ax.set_axis_labels("Number of Incidents Daily", "Density")
plt.title("Crime Distribution")

In [ ]:
col = sns.color_palette()

plt.figure(figsize=(16, 9))
data = sfcrime.groupby('incident_date').count().iloc[:, 0]
sns.kdeplot(data=data, shade=True)
plt.axvline(x=data.median(), ymax=0.95, linestyle='--', color=col[1])
plt.annotate(
    'Median: ' + str(data.median()),
    xy=(data.median(), 0.004),
    xytext=(200, 0.005),
    arrowprops=dict(arrowstyle='->', color=col[1], shrinkB=10))
plt.title('Distribution of number of incidents per day', fontdict={'fontsize': 16})
plt.xlabel('Incidents')
plt.ylabel('Density')
plt.legend().remove()
plt.show()

## Analyze Crime Data - Mapping

In [ ]:
# !pip3 install folium
 
import folium
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime
from folium import plugins

import json
import requests

### Source Geography

In [ ]:
#
# Source Geographies and save locally
# 

# San Francisco Analysis Neighborhood Geography
sfnhood_geojson = r"https://data.sfgov.org/api/geospatial/p5b7-5n3h?method=export&format=GeoJSON"
# Save locally
sfnhood_geojson_f = requests.get(sfnhood_geojson).json()
with open('sfnhood_geojson.json', 'w') as f:
    json.dump(sfnhood_geojson_f, f)

# San Francisco Police Disctrict Geography
sfpdistrict_geojson = r"https://data.sfgov.org/api/geospatial/wkhw-cjsf?method=export&format=GeoJSON"
# Save locally
sfpdistrict_geojson_f = requests.get(sfpdistrict_geojson).json()
with open('sfpdistrict_geojson.json', 'w') as f:
    json.dump(sfpdistrict_geojson_f, f)

# San Francisco Supervisor District Geography
sfsdistrict_geojson = r"https://data.sfgov.org/api/geospatial/f2zs-jevy?method=export&format=GeoJSON"
# Save locally
sfsdistrict_geojson_f = requests.get(sfsdistrict_geojson).json()
with open('sfsdistrict_geojson.json', 'w') as f:
    json.dump(sfsdistrict_geojson_f, f)

# San Francisco US Census Tract Geography
sftract_geojson = r"https://data.sfgov.org/api/geospatial/tmph-tgz9?method=export&format=GeoJSON"
# Save locally
sftract_geojson_f = requests.get(sftract_geojson).json()
with open('sftract_geojson.json', 'w') as f:
    json.dump(sftract_geojson_f, f)

# San Francisco US Census Tract Geography to SF Analysis Neighborhood
sfnhood_tract_geojson = r"https://data.sfgov.org/api/geospatial/sevw-6tgi?method=export&format=GeoJSON"
# Save locally
sfnhood_tract_geojson_f = requests.get(sfnhood_tract_geojson).json()
with open('sfnhood_tract_geojson.json', 'w') as f:
    json.dump(sfnhood_tract_geojson_f, f)

# Folium Colorscale
# https://github.com/python-visualization/folium/blob/v0.2.0/folium/utilities.py#L104

### Load (Saved) Geography

In [ ]:
#
# Load Geography from Local source
#
sfnhood_geojson = 'sfnhood_geojson.json'
sfpdistrict_geojson = 'sfpdistrict_geojson.json'
sfsdistrict_geojson = 'sfsdistrict_geojson.json'
sftract_geojson = 'sftract_geojson.json'
sfnhood_tract_geojson = 'sfnhood_tract_geojson.json'

In [ ]:
#
# San Francisco
# 
sfmap = folium.Map(
    location=[37.77,-122.42],
    tiles="Cartodb Positron",     
    zoom_start=13,
    width="%100",
    height="%100")

In [ ]:
#
# Map Style
#
style_function = lambda x: {'color':'black',
                            'weight':0.2, 
                            'fillColor':'lightblue', 
                            'fillOpacity':0.2
                            }

highlight_function = lambda x: {'color':'black',
                                'weight': 0.7,
                                'fillColor': 'lightgreen',
                                'fillOpacity': 0.50
                                }

In [ ]:
#
# SF Neighborhoods
# 
# Layers:   Crime Analysis Neighborhoods
#           Police Districts
#           Supervisor Districts
#           Census Tracts
#
sfmap = folium.Map(
                location=[37.763,-122.42],
                tiles="Cartodb Positron",     
                zoom_start=12,
                width="%100",
                height="%100")
                
folium.GeoJson(
            data = sfnhood_geojson, 
            style_function = style_function,
            highlight_function = highlight_function,
            name="Analysis Neighborhood").add_to(sfmap)

folium.GeoJson(
            data = sfpdistrict_geojson, 
            style_function = style_function,
            highlight_function = highlight_function,
            name="Police District").add_to(sfmap)

folium.GeoJson(
            data = sfsdistrict_geojson, 
            style_function = style_function,
            highlight_function = highlight_function,
            name="Supervisor District").add_to(sfmap)

folium.GeoJson(
            data = sfnhood_tract_geojson, 
            style_function = style_function,
            highlight_function = highlight_function,
            name="Census Tract").add_to(sfmap)

folium.LayerControl().add_to(sfmap)

sfmap

In [ ]:
#
# SF Neighborhood Population
#
sfmap = folium.Map(
    location=[37.77,-122.42],
    tiles="Cartodb Positron",     
    zoom_start=12,
    width="%100",
    height="%100")
    
folium.Choropleth(
    geo_data=sfnhood_geojson,
    data=sfnhood_acs5y2020,
    columns=['nhood', 'pop'],
    key_on='feature.properties.nhood',
    fill_color ='YlGnBu',
    fill_opacity = 0.7,
    line_opacity = 0.2,
    name ='Population',     
    legend_name = "Population Scale"
).add_to(sfmap)

sfmap

In [ ]:
#
# SF Neighborhood CHCI
#
sfmap = folium.Map(
    location=[37.77,-122.42],
    tiles="Cartodb Positron",     
    zoom_start=12,
    width="%100",
    height="%100")
    
folium.Choropleth(
    geo_data=sfnhood_geojson,
    data=sfnhood_acs5y2020,
    columns=['nhood', 'chci'],
    key_on='feature.properties.nhood',
    fill_color ='YlGnBu',
    fill_opacity = 0.7,
    line_opacity = 0.2,
    name ='CHCI',     
    legend_name = "CHCI Scale"
).add_to(sfmap)

sfmap

In [ ]:
#
# SF Neighborhood Crime Incidents
#
sfmap = folium.Map(
    location=[37.77,-122.42],
    tiles="Cartodb Positron",     
    zoom_start=12,
    width="%100",
    height="%100")
    
folium.Choropleth(
    geo_data=sfnhood_geojson,
    data=sfnhood_acs5y2020,
    columns=['nhood', 'incidents'],
    key_on='feature.properties.nhood',
    fill_color ='YlOrRd',
    fill_opacity = 0.7,
    line_opacity = 0.2,
    name ='Crime Incidents',     
    legend_name = "Crime Scale"
).add_to(sfmap)

sfmap

# SF Crime Heatmap by Time

## Modeling

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder

from statsmodels.formula.api import ols
import statsmodels.api as sm

In [ ]:
#
# Crime Rate - Dependant Variable
#
sfnhood_acs5y2020['crime_rate'] = sfnhood_acs5y2020['incidents'] / sfnhood_acs5y2020['pop']

In [ ]:
#
# Age Variables
#
sfnhood_acs5y2020['a75a'] = sfnhood_acs5y2020['a7584'] + sfnhood_acs5y2020['a85a']
sfnhood_acs5y2020['a65a'] = sfnhood_acs5y2020['a6574'] + sfnhood_acs5y2020['a75a']
sfnhood_acs5y2020['a60a'] = sfnhood_acs5y2020['a6064'] + sfnhood_acs5y2020['a65a']
sfnhood_acs5y2020['a5564'] = sfnhood_acs5y2020['a5559'] + sfnhood_acs5y2020['a6064']
sfnhood_acs5y2020['a3554'] = sfnhood_acs5y2020['a3544'] + sfnhood_acs5y2020['a4554']
sfnhood_acs5y2020['a2034'] = sfnhood_acs5y2020['a2024'] + sfnhood_acs5y2020['a2534']
sfnhood_acs5y2020['ab19'] = sfnhood_acs5y2020['ab5'] + sfnhood_acs5y2020['a59'] + sfnhood_acs5y2020['a1014'] + sfnhood_acs5y2020['a1519']
sfnhood_acs5y2020['ab15'] = sfnhood_acs5y2020['ab5'] + sfnhood_acs5y2020['a59'] + sfnhood_acs5y2020['a1014']
sfnhood_acs5y2020['ab10'] = sfnhood_acs5y2020['ab5'] + sfnhood_acs5y2020['a59']


In [ ]:
#
# Identify Variables
# 
df = sfnhood_acs5y2020

y = df['crime_rate']
x = df[[
    'chci',
    # "ab5",   # Under 5 years
    # "a59",   # 5 to 9 years
    # "a1014", # 10 to 14 years
    # "a1519", # 15 to 19 years
    "a2024", # 20 to 24 years
    "a2534", # 25 to 34 years
    "a3544", # 35 to 44 years
    "a4554", # 45 to 54 years
    # "a5559", # 55 to 59 years
    # "a6064", # 60 to 64 years
    # "a6574", # 65 to 74 years
    # "a7584", # 75 to 84 years
    # "a85a", # 85 years and over
    # 'a2034',
    # 'a3554',
    'a5564',
    'a6574',
    'a7584',
    'a85a',
    # 'a75a',
    # "mage", # Median age (years)
    'pop', 
    # 'poph_white', 
    'poph_black', 
    'poph_asian', 
    'poph_latino',
    'poph_aindian', 
    'poph_pacific', 
    'poph_other', 
    'poph_twoplus',
    'h_med_rooms',
    'h_rental_units',
    'h_rentel_size',
    'h_med_rent',
    'h_med_homeprice',
    'eco_clf', 
    'eco_unemp', 
    'eco_med_hincome', 
    'eco_poverty'
    ]] 
X = sm.add_constant(x)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
#
# Determine Variance
#

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["Variables"] = X.columns
vif.round(1)

In [ ]:
#
# Linear Regression Model
# 
eq01 = sm.OLS(y,X).fit()
print(eq01.summary())

In [ ]:
#
# Linear Regression Plots - Race
#
df = sfnhood_acs5y2020

# Remove outliers
# dindex = sfnhood_acs5y2020[sfnhood_acs5y2020['nhood'].isin(['Lincoln Park', 'Golden Gate Park', 'McLaren Park', 'The Farallones'])].index
# df = sfnhood_acs5y2020.drop(dindex)

fig, axes = plt.subplots(ncols = 4, nrows = 2, sharey = True, figsize=(16,9))

fig.suptitle('Linear Regression Race Variables')

sns.regplot(data = df, y="crime_rate", x="poph_latino", ax = axes[0, 0])
sns.regplot(data = df, y="crime_rate", x="poph_white", ax = axes[0,1 ])
sns.regplot(data = df, y="crime_rate", x="poph_black", ax = axes[0, 2])
sns.regplot(data = df, y="crime_rate", x="poph_aindian", ax = axes[0, 3])
sns.regplot(data = df, y="crime_rate", x="poph_asian", ax = axes[1, 0])
sns.regplot(data = df, y="crime_rate", x="poph_pacific", ax = axes[1, 1])
sns.regplot(data = df, y="crime_rate", x="poph_other", ax = axes[1, 2])
sns.regplot(data = df, y="crime_rate", x="poph_twoplus", ax = axes[1, 3])

In [ ]:
#
# Linear Regression Plots - Age
#
df = sfnhood_acs5y2020

# Remove outliers
# dindex = sfnhood_acs5y2020[sfnhood_acs5y2020['nhood'].isin(['Lincoln Park', 'Golden Gate Park', 'McLaren Park', 'The Farallones'])].index
# df = sfnhood_acs5y2020.drop(dindex)

fig, axes = plt.subplots(ncols = 4, nrows = 3, sharey = True, figsize=(16,12), )

fig.suptitle('Linear Regression Age Variables')

sns.regplot(data = df, y="crime_rate", x="ab5", ax = axes[0, 0])
sns.regplot(data = df, y="crime_rate", x="a59", ax = axes[0,1 ])
sns.regplot(data = df, y="crime_rate", x="a1014", ax = axes[0, 2])
sns.regplot(data = df, y="crime_rate", x="a1519", ax = axes[0, 3])
sns.regplot(data = df, y="crime_rate", x="a2024", ax = axes[1, 0])
sns.regplot(data = df, y="crime_rate", x="a2534", ax = axes[1, 1])
sns.regplot(data = df, y="crime_rate", x="a3544", ax = axes[1, 2])
sns.regplot(data = df, y="crime_rate", x="a4554", ax = axes[1, 3])
sns.regplot(data = df, y="crime_rate", x="a5559", ax = axes[2, 0])
sns.regplot(data = df, y="crime_rate", x="a6064", ax = axes[2, 1])
sns.regplot(data = df, y="crime_rate", x="a6574", ax = axes[2, 2])
sns.regplot(data = df, y="crime_rate", x="a7584", ax = axes[2, 3])

In [ ]:
#
# Golden Gate Park
# 
sfnhood_acs5y2020[['nhood','pop','incidents','crime_rate']].sort_values(['crime_rate'], ascending = False)

In [ ]:
#
# Identify Variables - Without Outlier Neighborhood (Parks)
# 
dindex = sfnhood_acs5y2020[sfnhood_acs5y2020['nhood'].isin(['Lincoln Park', 'Golden Gate Park', 'McLaren Park', 'The Farallones'])].index
df = sfnhood_acs5y2020.drop(dindex)

y = df['crime_rate']
x = df[[
    'chci',
    # "ab5",   # Under 5 years
    # "a59",   # 5 to 9 years
    # "a1014", # 10 to 14 years
    # "a1519", # 15 to 19 years
    "a2024", # 20 to 24 years
    "a2534", # 25 to 34 years
    "a3544", # 35 to 44 years
    "a4554", # 45 to 54 years
    # "a5559", # 55 to 59 years
    # "a6064", # 60 to 64 years
    # "a6574", # 65 to 74 years
    # "a7584", # 75 to 84 years
    # "a85a", # 85 years and over
    # 'a2034',
    # 'a3554',
    'a5564',
    'a6574',
    'a7584',
    'a85a',
    # 'a75a',
    # "mage", # Median age (years)
    'pop', 
    # 'poph_white', 
    'poph_black', 
    'poph_asian', 
    'poph_latino',
    'poph_aindian', 
    'poph_pacific', 
    'poph_other', 
    'poph_twoplus',
    'h_med_rooms',
    'h_rental_units',
    'h_rentel_size',
    'h_med_rent',
    'h_med_homeprice',
    'eco_clf', 
    'eco_unemp', 
    'eco_med_hincome', 
    'eco_poverty'
    ]] 
X = sm.add_constant(x)

In [ ]:
#
# Linear Regression Model - Without Outlier Neighborhood (Parks)
# 
eq01 = sm.OLS(y,X).fit()
print(eq01.summary())

In [ ]:
#
# Next Steps;
# Review other models to chose best performing
# Review crime forecasting by time and location
# Investigate Time Series forcasting
# ...